In [ ]:
import pandas as pd
from talib import RSI, BBANDS

DATA_STORE = 'sp.h5'

In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    data = store.get('prices_merged')

In [ ]:
# dropen von empty tickers
empty = []
for index, row in data.iterrows():
    if (row['prices'].empty):
        empty.append(index)
print(len(empty))
data = data.drop(empty, axis=0)

In [ ]:
# Berechnen der weekly returns
for index, row in data.iterrows():
    prices = row['prices']
    prices['weekly_return'] = prices['adjusted_close'].pct_change()
    data.loc[index]['prices'] = prices

In [ ]:
# Droppen von empty rows für beta
for index, row in data.iterrows():
    prices = row['prices']
    prices = prices.dropna(axis=0, how='any')
    data.loc[index]['prices'] = prices

In [ ]:
data.loc['SIAL']['prices'].tail()


In [ ]:
#  rsi berechnen
for index, row in data.iterrows():
    prices = row['prices']
    p = row['prices']['adjusted_close']
    try:
        rsi = RSI(p, 14)
    except:
        print(index)
    data.loc[index]['prices']['rsi'] = rsi

KG wird entfernt, da Daten fehlen

In [ ]:
data = data.drop('KG', axis=0)


In [ ]:
# #Alternativ bis hierher
# with pd.HDFStore(DATA_STORE) as store:
#     data = store.get('prices_merged')

# # Leere Ticker entfernen
# empty = [index for index, row in data.iterrows() if row['prices'].empty]
# data.drop(empty, axis=0, inplace=True)

# # Wöchentliche Renditen berechnen
# data['prices'] = data['prices'].apply(lambda prices: prices.assign(weekly_return=prices['adjusted_close'].pct_change()))

# # Zeilen mit leeren Beta-Werten entfernen
# data['prices'] = data['prices'].apply(lambda prices: prices.dropna(axis=0, how='any'))

# # RSI berechnen
# def calculate_rsi(prices):
#     rsi = RSI(prices['adjusted_close'], 14)
#     return prices.assign(rsi=rsi)

# data['prices'] = data['prices'].apply(calculate_rsi)


In [ ]:
# Bollinger Bänder berechnen

for index, row in data.iterrows():
    prices = row['prices']
    p = row['prices']['adjusted_close']
    if p.isna().all():  # Überprüfen, ob alle Werte in p NaN sind
        print(f"Skipping ticker {index} because all adjusted_close values are NaN.")
        continue

    u, m, l = BBANDS(p)
    data.loc[index]['prices']['bb_low'] = l 
    data.loc[index]['prices']['bb_mid'] = m 
    data.loc[index]['prices']['bb_upper'] = u 


In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('data_clean', data)

In [ ]:
# #neu
# # Laden der Daten
# DATA_STORE = 'sp.h5'
# with pd.HDFStore(DATA_STORE) as store:
#     data = store.get('prices_merged')

# # Löschen von doppelten Einträgen
# data.drop_duplicates(subset=['ticker', 'date'], inplace=True)

# # Entfernen von ungültigen Werten
# data.replace([np.inf, -np.inf], np.nan, inplace=True)
# data.dropna(inplace=True)

# # Ersetzen fehlender Werte durch den Durchschnittswert der benachbarten Einträge
# data.interpolate(method='linear', limit_direction='both', inplace=True)

# # Speichern der bereinigten Daten
# with pd.HDFStore(DATA_STORE) as store:
#     store.put('data_clean', data)
